# Import the Libraries

In [2]:
from binance import Client as c
from binance import BinanceSocketManager as bsm
from binance import ThreadedDepthCacheManager as tdcm

import mplfinance as mpf
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import math
import os
import time
from datetime import datetime, timedelta
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

# Access API, Extract USDT Crypto pairs

In [10]:
api_key = 'fx5ofaufVHZeHGCvMhA8pDt7oDaQ54tqIm89SbQXMg1kwSNjDgpfuAW9WWZy75UW'
api_secret = 'MmZB4hRimIG0pcfxQSmo8TdbE46SKlaQQO8vLDez3bTqU5dFgVwk7GGOHySlqarM'

client = c(api_key, api_secret)

In [11]:
#Access all ticket + immedate price
tickers = client.get_all_tickers()
#put them into a DF
tickers_df = pd.DataFrame(tickers)

#Filter out only ticker that is pair USDT
tickers_df = tickers_df[tickers_df['symbol'].str.contains('USDT')]

#Use Symbol as the index instead of default index
tickers_df.set_index('symbol', inplace =True) #inplace=True apply the function to the dataframe

#check the DF
tickers_df.shape

(425, 1)

In [12]:
# use this code to check for a specific pair
tickers_df.loc['BTCUSDT']

price    20760.74000000
Name: BTCUSDT, dtype: object

In [13]:
# Check he DF:
tickers_df.head()

,price
symbol,
BTCUSDT,20760.74000000
ETHUSDT,1564.79000000
BNBUSDT,290.70000000
BCCUSDT,448.70000000
NEOUSDT,8.65000000


In [30]:
# A fast way to Call the latest symbol 
tickers_df.index[tickers_df.shape[0]-1]

'APTUSDT'

In [70]:
# for i in range(1, len(data_all) + 1):          # Write each DataFrame to separate CSV
#     data_i = data_all['data' + str(i)]
#     data_i.to_csv('data' + str(i) + '.csv')

# Use this ?? to get doc help
#historical = client.get_historical_klines


In [ ]:
#HISTORICAL DATA IS K-line
for i in range(0, len(tickers_df)-1):
    
    # Access all the Kline data of a symbol from today to 1000 days ago (1/2020)
    # Max Limit is 1000 lines
    historical1 = client.get_historical_klines(tickers_df.index[i], client.KLINE_INTERVAL_1DAY, limit =1000)
    
    #Access date 2000 days ago:
    date_2K_ago = str(datetime.now() + timedelta(days=-2000))
    # Access all the Kline data of a symbol from 1000 days ago (1/2020) to 2000 days ago (5/2017)
    # Max Limit is 1000 lines
    historical2 = client.get_historical_klines(tickers_df.index[i], client.KLINE_INTERVAL_1DAY, start_str=date_2K_ago, limit =1000)
    
    #Append data from 2000 days ago till now
    historical = historical1 + historical 2
    
    # Put the data into a DF
    hist_df = pd.DataFrame(historical)

    # Name the columns then convert the Open and Close time to second
    hist_df.columns = ['Open time', 'Open', 'High', 'Low', 'Close','Volume', 'Close time', 'Quote asset volume', 'Number of trades','Taker buy base asset volume', 'Taker buy quote asset volume', 'ignore']
    hist_df['Open time'] = pd.to_datetime(hist_df['Open time']/ 1000, unit ='s')
    hist_df['Close time'] = pd.to_datetime(hist_df['Close time']/ 1000, unit ='s')

    # Convert all numeric columns into the right format: float64
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Taker buy base asset volume', 'Taker buy quote asset volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

    # Add a column with a symbol of the crypto
    hist_df['symbol'] = tickers_df.index[i]
    
    #Save to .csv 
    path = r'C:\Users\ethai\OneDrive\1.1. Education\1. BrainStation\0. Capstone\data\ ' #super important to have 'r' and '\ ' -backward slash with space in the end
    hist_df.to_csv(path + tickers_df.index[i]+'.csv')